# Topic Modeling

In this section, we used the preprocessed dataset from feature_engineering.ipynb to model coherent topics.

In [ ]:
#importing necessary packages
import pandas as pd
import matplotlib.pyplot as plt
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim_models

In [2]:
#loading in preprocessed data
trigram = pd.read_csv('data/trigram.csv')

In [3]:
#cleaning tweets
trigram_sentences = trigram.lemma_words_trigram.apply(lambda x: x.split())

#creating dictionary for lda model
id2word_trigrams = corpora.Dictionary(trigram_sentences)

#creating corpus for lda model
texts_trigrams = list(trigram_sentences)
corpus_trigram = [id2word_trigrams.doc2bow(x) for x in texts_trigrams]

In [4]:
#creating trigram model
lda_model_trigram_three_topics = gensim.models.ldamodel.LdaModel(corpus=corpus_trigram, id2word=id2word_trigrams, num_topics=3, random_state=100, update_every=1, chunksize=100, passes=10, alpha='auto', per_word_topics=True)

In [5]:
import pyLDAvis
import pyLDAvis.gensim_models

#visualizing model
pyLDAvis.enable_notebook()
vis_3 = pyLDAvis.gensim_models.prepare(lda_model_trigram_three_topics, corpus_trigram, id2word_trigrams)
vis_3

C:\Users\mikey\anaconda3\envs\contest_env\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.360693 -0.040108       1        1  36.365214
0      0.215240 -0.290895       2        1  31.832999
2      0.145454  0.331002       3        1  31.801787, topic_info=         Term          Freq         Total Category  logprob  loglift
356    crypto  32481.000000  32481.000000  Default  30.0000  30.0000
85        get  31271.000000  31271.000000  Default  29.0000  29.0000
182     think  31428.000000  31428.000000  Default  28.0000  28.0000
309    people  30109.000000  30109.000000  Default  27.0000  27.0000
105        go  23576.000000  23576.000000  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
478      hold   5651.059549   5651.944755   Topic3  -5.4075   1.1455
770       ask   5338.696249   5339.540270   Topic3  -5.4643   1.1455
1229  account   5459.831447   5460.701061   Topic3  -5.4419   1.1455
2495   please   6990.397060   6991.577720   Topic3  -5.1948   1.1455
1640  company   6087.325768   6091.839117   Topic3  -5.3331   1.1449

[128 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
1229      3  0.999872   account
239       3  0.999926  actually
405       1  0.000582      also
405       2  0.999430      also
128       2  0.999949       amp
...     ...       ...       ...
480       3  0.999901      well
65        1  0.999974      work
338       1  0.999880     world
216       3  0.999928      year
2013      1  0.999782       yes

[112 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 3])

In [6]:
#evaluating coherence of LDA model
coherence_model_trigram_three_topics = CoherenceModel(model=lda_model_trigram_three_topics, texts=texts_trigrams, dictionary=id2word_trigrams, coherence='c_v')
coherence_model_trigram_three_topics.get_coherence()

0.21741394603184763

### Coherence score is not good but substantially better than baseline score of 9%.

In [7]:
#appending probabilities of each tweet
vecs_0 = []
vecs_1 = []
vecs_2 = []
for i in range(len(trigram)):
    top_topics = lda_model_trigram_three_topics.get_document_topics(corpus_trigram[i], minimum_probability=0.0)
    vecs_0.append(top_topics[0][1])
    vecs_1.append(top_topics[1][1])
    vecs_2.append(top_topics[2][1])


#creating dataframe
topic_0 = pd.Series(vecs_0, index=trigram.index)
topic_1 = pd.Series(vecs_1, index=trigram.index)
topic_2 = pd.Series(vecs_2, index=trigram.index)

topic_model_df = pd.concat([trigram, topic_0, topic_1, topic_2], axis=1)

#saving dataframe for model testing
topic_model_df.to_csv('data/topic_model_dataframe.csv')